---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [73]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

#pd.set_option('display.max_columns', 50)
#pd.set_option('display.max_rows', 100)

def blight_model():
    # ------------- Load data -------------
    df_train = pd.read_csv('train.csv', encoding='latin1')
    df_test = pd.read_csv('test.csv', encoding='latin1')
    df_addresses = pd.read_csv('addresses.csv', encoding='latin1')
    df_latlons = pd.read_csv('latlons.csv', encoding='latin1')

    # ------------- Join different datasets -------------
    new_df = pd.merge(df_train, df_addresses,  how='left', left_on=['ticket_id'], right_on = ['ticket_id'])
    new_df2 = pd.merge(df_test, df_addresses,  how='left', left_on=['ticket_id'], right_on = ['ticket_id'])
    df_train = pd.merge(new_df, df_latlons,  how='left', left_on=['address'], right_on = ['address'])
    df_test = pd.merge(new_df2, df_latlons,  how='left', left_on=['address'], right_on = ['address'])

    # ------------- Pre-processing -------------
    # Fill NA for rows where there are no lat / lon
    df_train.lat.fillna(method='pad', inplace=True)
    df_train.lon.fillna(method='pad', inplace=True)
    df_test.lat.fillna(method='pad', inplace=True)
    df_test.lon.fillna(method='pad', inplace=True)

    # Remove Null values (not taken into account for the tests)
    df_train = df_train[df_train.compliance.notnull()]
    df_train = df_train[df_train.hearing_date.notnull()]

    # Add features
    X_train = df_train[['agency_name','disposition','fine_amount','discount_amount','judgment_amount', 'lat', 'lon', 'admin_fee', 'state_fee', 'late_fee', 'clean_up_cost']]
    y_train = df_train[['compliance']]

    X_test = df_test[['agency_name','disposition','fine_amount','discount_amount','judgment_amount', 'lat', 'lon', 'admin_fee', 'state_fee', 'late_fee', 'clean_up_cost']]

    # create new features based on discrete values columns
    X_train = pd.get_dummies(X_train, columns = ['agency_name', 'disposition'])
    X_test = pd.get_dummies(X_test, columns = ['agency_name', 'disposition'])

    # Add missing field as columns in each dataframe
    X_train['disposition_Responsible (Fine Waived) by Admis'] = 0
    X_train['disposition_Responsible - Compl/Adj by Default'] = 0
    X_train['disposition_Responsible - Compl/Adj by Determi'] = 0
    X_train['disposition_Responsible by Dismissal'] = 0

    X_test['agency_name_Health Department'] = 0
    X_test['agency_name_Neighborhood City Halls'] = 0

    # split data for training
    #X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, random_state=0)

    # ------------- Scale data -------------
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # ------------- Train Algo : Random Forest -------------
    clf = RandomForestClassifier(n_estimators=350, random_state=0, max_features='auto', max_depth=12, criterion='entropy', class_weight="balanced").fit(X_train_scaled, y_train)

    # Proba
    y_proba = clf.predict_proba(X_test_scaled)[:,1]

    # Integrate with reloaded test data
    test_df = pd.read_csv('test.csv', encoding = "latin1")
    test_df['compliance'] = y_proba
    test_df.set_index('ticket_id', inplace=True)

    test_df.compliance
    
    return test_df.compliance

In [74]:
blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


ticket_id
284932    0.606926
285362    0.528391
285361    0.657357
285338    0.737017
285346    0.728828
285345    0.696439
285347    0.645090
285342    0.853905
285530    0.414069
284989    0.625360
285344    0.675136
285343    0.615804
285340    0.660724
285341    0.649621
285349    0.722416
285348    0.644604
284991    0.624983
285532    0.509841
285406    0.641980
285001    0.465263
285006    0.218629
285405    0.498239
285337    0.559541
285496    0.723939
285497    0.756502
285378    0.598039
285589    0.468136
285585    0.727823
285501    0.711718
285581    0.548679
            ...   
376367    0.235446
376366    0.715549
376362    0.549173
376363    0.426558
376365    0.235446
376364    0.715549
376228    0.824522
376265    0.696036
376286    0.950983
376320    0.692715
376314    0.687423
376327    0.961485
376385    0.909898
376435    0.763739
376370    0.931864
376434    0.517325
376459    0.628835
376478    0.047232
376473    0.687437
376484    0.275955
376482    0.218038
37